In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum as Fsum, udf, desc, asc
from pyspark.sql.types import StringType, IntegerType

import datetime

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
session = SparkSession.builder.appName("Assignment").getOrCreate()

23/01/16 10:50:46 WARN Utils: Your hostname, franklinobasy resolves to a loopback address: 127.0.1.1; using 172.24.108.31 instead (on interface eth0)
23/01/16 10:50:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/16 10:50:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/01/16 10:50:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
path = "data/sparkify_log_small.json"
user_log = session.read.json(path)

# 1. Which page did user id "" (empty string) NOT visit?

In [4]:
user_log.describe().show()

+-------+-----------------+----------+---------+------+------------------+--------+-----------------+-----+------------+------+-------+--------------------+------------------+--------+-----------------+-------------------+--------------------+------------------+
|summary|           artist|      auth|firstName|gender|     itemInSession|lastName|           length|level|    location|method|   page|        registration|         sessionId|    song|           status|                 ts|           userAgent|            userId|
+-------+-----------------+----------+---------+------+------------------+--------+-----------------+-----+------------+------+-------+--------------------+------------------+--------+-----------------+-------------------+--------------------+------------------+
|  count|             8347|     10000|     9664|  9664|             10000|    9664|             8347|10000|        9664| 10000|  10000|                9664|             10000|    8347|            10000|         

In [5]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [16]:
# user_log.select("page").dropDuplicates().sort("page").show()
page = user_log.select("page").dropDuplicates().sort("page")

In [17]:
page.show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



In [18]:
empty_string_user_page = user_log.select("page").where(user_log.userId == "").dropDuplicates().sort("page")

In [26]:
empty_string_user_page = empty_string_user_page.rdd.flatMap(lambda x: x).collect()

## Answer

In [34]:
user_log.select("page").where(~user_log.page.isin(empty_string_user_page)).dropDuplicates().sort("page").show()

+----------------+
|            page|
+----------------+
|       Downgrade|
|           Error|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



# 2. How many female users do we have in the data set?

In [36]:
user_log.select("UserId").where(user_log.gender == "F").dropDuplicates().count()

462

# 3. How many songs were played from the most played artist?

In [37]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [47]:
user_log.select("artist").groupby("artist").count().orderBy("count", ascending=False).show()

+--------------------+-----+
|              artist|count|
+--------------------+-----+
|                null| 1653|
|            Coldplay|   83|
|       Kings Of Leon|   69|
|Florence + The Ma...|   52|
|            BjÃÂ¶rk|   46|
|       Dwight Yoakam|   45|
|       Justin Bieber|   43|
|      The Black Keys|   40|
|         OneRepublic|   37|
|        Jack Johnson|   36|
|                Muse|   36|
|           Radiohead|   31|
|        Taylor Swift|   29|
|Barry Tuckwell/Ac...|   28|
|          Lily Allen|   28|
|               Train|   28|
|           Metallica|   27|
|           Daft Punk|   27|
|          Nickelback|   27|
|          Kanye West|   26|
+--------------------+-----+
only showing top 20 rows



In [42]:
user_log.select(["song"]).where(".groupby(["artist", "song"]).count().orderBy("count", ascending=False).show()

+--------------------+--------------------+-----+
|              artist|                song|count|
+--------------------+--------------------+-----+
|                null|                null| 1653|
|       Dwight Yoakam|      You're The One|   45|
|            BjÃÂ¶rk|                Undo|   44|
|Barry Tuckwell/Ac...|Horn Concerto No....|   28|
|         OneRepublic|             Secrets|   25|
|Florence + The Ma...|Dog Days Are Over...|   23|
|       Kings Of Leon|             Revelry|   22|
|            Harmonia|       Sehr kosmisch|   22|
|            Coldplay|              Yellow|   21|
|       Kings Of Leon|        Use Somebody|   19|
|    Five Iron Frenzy|              Canada|   17|
|            Tub Ring|             Invalid|   16|
|           Sam Cooke|    Ain't Misbehavin|   16|
|  Charttraxx Karaoke|           Fireflies|   13|
|        Travie McCoy|Billionaire [feat...|   13|
|       Justin Bieber|    Somebody To Love|   13|
| Angels and Airwaves|            The Gift|   12|


# 4. How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.

In [4]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
user_log.select(["song", "length"]).take(20)

[Row(song='Christmas Tears Will Fall', length=232.93342),
 Row(song='Cheryl Tweedy', length=195.23873),
 Row(song='Good Girls Go Bad (Feat.Leighton Meester) (Album Version)', length=196.20526),
 Row(song="Don't See The Point", length=405.99465),
 Row(song=None, length=None),
 Row(song=None, length=None),
 Row(song='Smoke Buddah', length=154.53995),
 Row(song='On My Own', length=402.93832),
 Row(song=None, length=None),
 Row(song=None, length=None),
 Row(song="Heart Of The City (Ain't No Love)", length=223.26812),
 Row(song='Bring Me To Life', length=237.11302),
 Row(song='Laura', length=218.01751),
 Row(song='Here Without You', length=237.13914),
 Row(song='This Old House w/ When The Saints Medley', length=191.68608),
 Row(song='Follow Me (Club Mix)', length=383.58159),
 Row(song=None, length=None),
 Row(song='Undo', length=348.57751),
 Row(song='Sheebeg And Sheemore', length=146.36363),
 Row(song='Far Away (Album Version)', length=238.18404)]

In [10]:
home_page_listen = user_log.select("page").where((user_log.page == "Home") & (user_log.length != None)).count()
all_home_page = user_log.select("page").where(user_log.page == "Home").count()


0.0
0
1126


In [ ]:
sum_ = user_log.select("length").where("page" == "Home").filter(user_log.length.isNotNull()).sum()
count = user_log.select("length").where("page" == "Home").filter(user_log.length.isNotNull()).count()